In [3]:
import pandas as pd
import numpy as np
import ast
import umap
import plotly.express as px
import plotly.graph_objs as go

In [27]:
embeddings = np.load('../data/embeddings.npy')
books = pd.read_csv('books_cleaned.csv')
books['tags'] = books['tags'].apply(ast.literal_eval)

In [28]:
u_2d = umap.UMAP(n_components=2, n_neighbors=250)
umap_2d = u_2d.fit(embeddings)
umap_coords = umap_2d.transform(embeddings)

/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/adam-kotik/anaconda3/envs/bakalarka/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [55]:
P = 2
scores_0 = []
for emb in embeddings:
    score = np.exp(-(1 - np.dot(embeddings[75], emb)) * P)
    scores_0.append(score)

scores_0 = np.array(scores_0)
print(len(scores_0))
scores_0 = np.divide(scores_0, np.sum(scores_0))
print(len(scores_0))

9214
9214


In [56]:
scores_0 = np.divide(scores_0, np.max(scores_0))
print(max(scores_0))
print(min(scores_0))

1.0
0.12380869656274625


In [ ]:
random_colors = np.random.rand(umap_coords.shape[0])
print(max(random_colors))
print(min(random_colors))
print(type(random_colors))
print(type(scores_0))

0.9999944127598507
0.0005708705558519211
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
12


In [31]:
book_75_coord = umap_coords[75]

In [58]:
trace = go.Scatter(
    x=umap_coords[:, 0],
    y=umap_coords[:, 1],
    mode='markers',
    marker=dict(
        size=8,
        color=scores_0,  # Use scores_0 for color intensity
        colorscale='Viridis',  # Choose a color scale (you can change it)
        showscale=True,  # Show color scale legend
        colorbar=dict(title="Scores Intensity")
    )
)

# Highlight the book with index 75
highlight_trace = go.Scatter(
    x=[book_75_coord[0]],
    y=[book_75_coord[1]],
    mode='markers',
    marker=dict(
        size=12,
        color='red',  # Highlight color for book 75
        symbol='star'
    )
)

# Combine traces for the plot
data = [trace, highlight_trace]

# Create layout
layout = go.Layout(
    title="UMAP Visualization of Books with Score Intensity",
    xaxis=dict(title="UMAP 1"),
    yaxis=dict(title="UMAP 2"),
    hovermode='closest'
)

# Create the figure and show it
fig = go.Figure(data=data, layout=layout)
fig.show()